# Orchestrate Multiple-Agents for High-Quality Research with AutoGen and LangChain Tools

This notebook showcases how to harness the combined power of **AutoGen** and **LangChain** tools to automate and elevate deep research workflows. At its core, the system coordinates a network of specialized agents—each executing a distinct role in the research and report generation workflow. Together, these agents collect data, analyze findings, and produce polished, insight-driven reports.


[Open in Colab](https://colab.research.google.com/github/miztiik/taars/blob/master/notebooks/deepresearch_w_autogen_langchain_tools.ipynb) <a href="https://colab.research.google.com/github/miztiik/taars/blob/master/notebooks/deepresearch_w_autogen_langchain_tools.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 🧠 Agent Roles Overview

- **🧭 Planner Agent**  
  Defines the research scope, objectives, and key questions.  
  → Anchors the process by structuring it around a well-defined `__TASK_QUERY`.

- **🔍 Researcher Agent**  
  Gathers information using tools like `wiki_search` and `duckduckgo_search`.  
  → Supplies the system with relevant, factual data.

- **🧪 Critic Agent**  
  Reviews the research plan, gathered evidence, and drafted reports.  
  → Provides feedback to enforce analytical rigor and clarity.

- **✍️ Editor Agent**  
  Refines the final report's language, structure, and presentation.  
  → Ensures clarity, polish, and alignment with stakeholder expectations.


In [ ]:
%pip install -U -q autogen-agentchat 
%pip install -U -q autogen-ext

%pip install -U -q langchain 
%pip install -U -q langchain-community
%pip install lxml
%pip install ddgs


In [ ]:
from pathlib import Path
from loguru import logger
import sys

# import logging
# logger = logging.getLogger("")


# # Remove all existing handlers
# for handler in logger.handlers[:]:
#     logger.removeHandler(handler)

# REMOVE ALL EXISTING HANDLERS for LOGURU
if len(logger._core.handlers) > 1:
    logger.configure(handlers=[])

logger.add(
    sys.stderr,
    format="{time:HH:mm:ss:SSS} | {level} | {name}:{line} | {message}",
    level="INFO",
    colorize=True,
)

# Configure file logging with better settings

notebook_dir = Path.cwd()
log_file = notebook_dir / "logs" / "llm_execution.log"
log_file.parent.mkdir(exist_ok=True)
logger.add(
    log_file,
    format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level: <8} | {name}:{function}:{line} | {message}",
    level="DEBUG",
    rotation="10 MB",
    retention="7 days",
    compression="zip",
    enqueue=True,
)
logger.info(f"✅ Logging configured - File: {log_file}")


In [ ]:
import os
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelInfo


# Confirm the API key is set
assert os.environ["GEMINI_API_KEY"], "GEMINI_API_KEY is not set"
assert os.environ["GEMINI_MODEL_NAME"], "GEMINI_MODEL_NAME is not set"
assert os.environ["RATE_LIMIT_GEMINI_RPM"], "RATE_LIMIT_GEMINI_RPM is not set"
assert os.environ["RATE_LIMIT_GEMINI_RPD"], "RATE_LIMIT_GEMINI_RPD is not set"
assert os.environ["GEMINI_BASE_URL"], "GEMINI_BASE_URL is not set"


model_info = ModelInfo(
    vision=True,
    function_calling=True,
    json_output=True,
    family=os.environ["GEMINI_MODEL_NAME"],
    structured_output=True,
)

model_client = OpenAIChatCompletionClient(
    model=os.environ["GEMINI_MODEL_NAME"],
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=os.environ["GEMINI_BASE_URL"],
    model_info=model_info,
)


In [ ]:
import asyncio, json, os
from regex import D
import tenacity
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.messages import StopMessage
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage


from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool
from ddgs import DDGS


# assert os.environ["OPENAI_API_KEY"], "OPENAI_API_KEY is not set"
# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o", api_key=os.environ["OPENAI_API_KEY"]
# )

# Tool definitions
wiki_api = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=2000)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api)


def wiki_search(q: str) -> dict:
    """Return structured output including text and source."""
    try:
        result = wiki_tool.run(q)
        return {"text": result, "source": "Wikipedia"}
    except Exception as e:
        return {"text": f"Error: {str(e)}", "source": None}


@tool
def duckduckgo_search(q: str) -> dict:
    """Search the web using DuckDuckGo for information."""
    try:
        results = DDGS().text(q, region="us-en", safesearch="off", max_results=12)
        return {
            "text": results,
            "source": "DuckDuckGo",
            "query": q,
            "results_found": len(results),
        }
    except Exception as e:
        return {"text": f"Error: {str(e)}", "source": None}


# ---------- Agents ----------
supervisor = AssistantAgent(
    name="Supervisor",
    description="Oversees the process, approves final output, requests retries if needed.",
    model_client=model_client,
    system_message="You monitor progress and decide when to terminate the task.",
)

planner = AssistantAgent(
    name="Planner",
    model_client=model_client,
    system_message=(
        "You are a meticulous planner. Create a clear, step-by-step plan for the request. "
        "End with PLAN_COMPLETE."
    ),
    description="Creates detailed step-by-step plans for tasks",
)

researcher = AssistantAgent(
    name="Researcher",
    description="Collects factual evidence using wiki_search, duckduckgo_search, produces text drafts.",
    model_client=model_client,
    tools=[wiki_search, duckduckgo_search],
    system_message="Research and produce a concise draft on the given topic using facts.",
)

critic = AssistantAgent(
    name="Critic",
    model_client=model_client,
    system_message=(
        """
You are a domain-agnostic Critic Agent. Your role: produce a concise, evidence-based critique of the provided artifact (plan, report, code, design, or prompt). Follow this exact process and style rules.

Process:
1. IDENTIFY: State artifact type, asserted goal, and audience (1 line).
2. SUMMARIZE: Give a 1–2 line neutral summary of content and intent.
3. STRENGTHS: List up to 5 concise strengths with line/section references.
4. ISSUES: For each issue, provide:
   - Title (one line)
   - Severity: {Critical, Major, Minor}
   - Evidence: exact quote / line numbers / code snippet
   - Impact: one sentence on consequences
   - Fix: Suggest corrective actions.
   - Tests to verify fix (if applicable)
5. PRIORITY LIST: Rank recommended fixes (highest impact first).
6. VERDICT: Single-line overall recommendation (Accept / Accept with changes / Major revision / Reject) and confidence level (High/Med/Low).
7. CLARIFY: If essential facts are missing for critical judgments, list the minimal questions required.
8. VERACITY: Assess the truthfulness and reliability of the information presented. Request sources or evidence for claims. Preferred format: [source](URL).
9. TRADEOFFS: Implore second-order consequences for unverified assumptions.

Style rules:
- Use active voice and action verbs.
- Keep each bullet ≤ 20 words.
- Use numbered lists and bullets.
- Cite exact lines or code references for claims.
- Challenge assumptions and seek clarification.
- End with a the author can act on.

Length limits:
- Summary: ≤ 40 words.
- Strengths: ≤ 5 bullets.
- Issues: ≤ 10 bullets.
- Overall critique: ≤ 500 words.

Tone: professional, concise, constructive.
"""
        "Do NOT APPROVE without addressing all concerns."
        "Respond with 'APPROVED: [...]' when your feedback is addressed."
    ),
    description="Reviews and provides constructive criticism; outputs 'APPROVED: [...]' when ready.",
)

editor = AssistantAgent(
    name="Editor",
    description="Formats an approved draft into Markdown, and appends 'TERMINATE'.",
    model_client=model_client,
    system_message=(
        "Format Critic approved content output in Markdown format."
        "Focus on clarity, coherence, and conciseness."
        "Use bullets and tables where appropriate."
        "append 'TERMINATE' to signal completion."
    ),
)

# ---------- Selector prompt with few-shot examples ----------
selector_prompt = """You are the selector agent. Choose exactly one agent name from {participants} to speak next based on the conversation history and their roles.

Few-shot examples:

Example 1:
History: Researcher produced a draft; Critic says 'Needs more sources and clarity'.
Selected agent: Researcher

Example 2:
History: Researcher produced a draft; Critic says 'APPROVED'.
Selected agent: Editor

Roles:
{roles}

Conversation history:
{history}

Rules:
- Pick the agent whose role most advances completion.

Return only the agent name (no extra text).
"""


# ---------- Termination callable ----------
max_messages = 25
txt_termination = TextMentionTermination("TERMINATE")
termination_condition = (
    MaxMessageTermination(max_messages=max_messages) | txt_termination
)

# ---------- Build SelectorGroupChat ----------
team = SelectorGroupChat(
    participants=[supervisor, planner, editor, researcher, critic],
    model_client=model_client,
    selector_prompt=selector_prompt,
    termination_condition=termination_condition,
)


# ---------- Selector validation ----------
def validate_selector_choice(raw_choice, participants):
    """Ensure selector returns a canonical agent name."""
    choice = raw_choice.strip().splitlines()[0].strip()
    for p in participants:
        if choice.lower() == p.name.lower():
            return p.name
    return None


In [ ]:
import nest_asyncio
nest_asyncio.apply()


In [ ]:
@tenacity.retry(
    wait=tenacity.wait_exponential(multiplier=1, min=1, max=20),
    stop=tenacity.stop_after_attempt(2),
    retry=tenacity.retry_if_exception_type(Exception),
)
async def run_task(task_text: str):
    final_report = None
    try:
        async for ev in team.run_stream(task=task_text):
            logger.info(ev)
            if await termination_condition([ev]):
                if isinstance(ev, TextMessage):
                    final_report = ev.content.split("TERMINATE", 1)[0].strip()
                    break
        # Save final state
        state = await team.save_state()
        with open("team_state.json", "w") as f:
            json.dump(state, f)
    except Exception as e:
        print("Exception during run:", e)
        # Save state on failure
        state = await team.save_state()
        with open("team_state.json", "w") as f:
            json.dump(state, f)
        raise
    return final_report


async def resume_from_saved_state():
    with open("team_state.json", "r") as f:
        saved = json.load(f)
    await team.load_state(saved)
    async for ev in team.run_stream():
        logger.info(ev)


if __name__ == "__main__":
    __TASK_QUERY = "AI , Macro economic pressure, tarrifs on indian power sector"
    output = asyncio.run(run_task(__TASK_QUERY))
    if output:
        print("\nFinal Report:\n", output)
    else:
        print("Task did not complete successfully.")
